<center> <h1>Geleneksel Doğal Dil İşleme Yöntemleri ve Uygulamaları</h1> </center>

<a class = 'anchor' id = '0.'></a><h2>İçindekiler</h2>
<font size = 3>
* [1. Çalışmanın Özeti](#1.)
* [2. Cümlenden İsim Sözcüklerini Çıkarma](#2.)
* [3. Metin Benzerliği Hesaplama](#3.)
* * [3.1. Kosine Benzerlik Hesaplama](#3.1.)
* * [3.2. Phonetic Benzerlik Eşleşme](#3.2.)
* [4. Cümledeki Sözcükelerin Etiketlenmesi](#4.)
* [5. Cümledeki Sözcüklerden Varlıkları Çıkarma (Named Entity Recognition)](#5.)
* [6. Başlık Belirleme](#6.)
* [7. Metin Sınıflandırma](#7.)
* [8. Duygu Analizi (TextBlob Kütüphanesiyle)](#8.)
* [9. Belirsizlik Giderme](#9.)
* [10. Konuşmayı Metne Dönüştürme](#10.)
* [11. Metni Konuşmaya Dönüştürme](#11.)
* [12. Tercüme Yapma](#12.)
</font>

In [1]:
import numpy as np
import pandas as pd
import gensim
from gensim import corpora
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='1.'></a>1. Çalışmanın Özeti

Bu çalışmada geleneksel doğal dil işleme yöntemleri ve bu yöntemler kullanılarak uygulamalar geliştirilecektir. Üzerinde çalışılacak yöntemler aşagıdaki gibidir:

    * Cümleden isim çıkarma
    * Metin benzerliği
    * Cümledeki sözcüklerin etiketlenmesi
    * Named Entitiy Recognition
    * Başlık modelleme
    * Metin sınıflandırma
    * Duygu analizi
    * Kelime anlam karmaşıklığı
    * Konuşma tanıma ve konuşmadan metne dönüşüm
    * Metinden konuşmaya
    * Dil tespiti ve tercübe

[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='2.'></a>2. Cümlenden İsim Sözcüklerini Çıkarma

Cümleden isim sözcüklerinin çıkarmak için textblob kütüphanesinden TextBlob nesnesi kullanılacaktır. TextBlob nesnesine sözcük girdi olarak verilecek ve oluşturulan nesnenin <b>noun_phareses </b> özelliğinden isim sözcükleri alılacaktır.

Uyarı : Bu özellik Türkçe için bu kütüphanede yoktur. Hangi kütüphanelerde Türkçe cümleden isim çıkarma özelliği olduğu araştırılacaktır

In [2]:
blob = TextBlob('Serkan is working on natural language processing subjects')

In [3]:
for np in blob.noun_phrases:
    print(np)

serkan
natural language processing subjects


[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='3.'></a> 3. Metin Benzerliği Hesaplama

İki metin arasındaki benzerliği bulmak çoğu zaman ihtiyaç duyulan bir hesaplamadır. İki metin arasındaki benzerliği bulmak için farklı hesaplama metrikleri vardır:

* <b> Cosine benzerliği:</b> İki vektör arasındaki cosine açısını hesaplar.
* <b> Jaccard benzerliği: </b> İki metin arasındaki ortak kelimelerin toplam kelimeye bölümüyle elde edilir.
* <b> Levenshtein benzerliği :</b> A dizgisini B dizgisine dönüştürmek için gerekli minumum değişiklik, ekleme, düzeltmeyi hesaplar
* <b> Hamming bezerliği:</b> Aynı uzunluğa sahip iki dizge arasındaki aynı pozisyondaki sembol farklılıklarını hesaplar


[İçindekiler Menüsüne Git](#0.)
## <a class='anchor' id ='3.1.'></a> 3.1. Kosine Benzerlik Hesaplama

In [4]:
sorular = ['İzmir’de kentsel dönüşüm nerelerde yapılmaktadır?',
          'İzmir Büyükşehir Belediyesinin süt soğutma tankı desteklerinden nasıl faydalanabilirim?',
          'Spor okulları hakkında nasıl bilgi alabilirim?',
          'Haşere ilaçlaması için nereye başvurmalıyım?',
          'Bakamayacağım köpeğimi belediyeye ait bir barınağa teslim edebilir miyim?']

tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer.fit(sorular)
sorular_tfidf = tfidfVectorizer.transform(sorular)
print('sorular_tfidf.shape:', sorular_tfidf.shape)

sorular_tfidf.shape: (5, 33)


In [5]:
yeni_soru = ['belediye süt bankası soğutma için', 'ilaçlaması hakkında nereye']
yeni_soru_tfidf = tfidfVectorizer.transform(yeni_soru)
print(cosine_similarity(yeni_soru_tfidf, sorular_tfidf))

[[0.         0.40075818 0.         0.25819889 0.        ]
 [0.         0.         0.2428734  0.51639778 0.        ]]


[İçindekiler Menüsüne Git](#0.)
## <a class='anchor' id ='3.2.'></a> 3.2. Phonetic Benzerlik Eşleşme

Phonetic eşleşme yazımı benzer kelimeleri ortak kodlarla temsil eden benzerlik eşleştirme yöntemidir.

In [6]:
# fuzzy yükleme başarısız olduğu için kodları yazmakla yetiniyorum
# import fuzzy
#soundex = fuzzy.Soundex()
#soundex('natural') #N364 kodunu alacaktır
#soundex('natuaral') # N364 kodunu alacaktır

[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='4.'></a>4. Cümledeki Sözcüklerin Etiketlenmesi (Tagging Part of Speech (POS))

POS; Varlık Tanıma (Named Entity Recognition), Soru/Cevaplama (Question/Answering), Kelime Anlam Belirsizliği (Word Sense Disambiguation) ve Duygu Analizi gibi doğal dil işleme problemlerinin çözümünün temellerini oluşturur.


In [7]:
stopwords_eng = set(stopwords.words('english'))

text = 'I am learnign natural language processing with reading Python book written by Akshay'

word_token = word_tokenize(text)
print(word_token)
word_token = [word for word in word_token if not word in stopwords_eng]
print(word_token)
pos_tag = nltk.pos_tag(word_token)
print(pos_tag)

['I', 'am', 'learnign', 'natural', 'language', 'processing', 'with', 'reading', 'Python', 'book', 'written', 'by', 'Akshay']
['I', 'learnign', 'natural', 'language', 'processing', 'reading', 'Python', 'book', 'written', 'Akshay']
[('I', 'PRP'), ('learnign', 'VBP'), ('natural', 'JJ'), ('language', 'NN'), ('processing', 'NN'), ('reading', 'VBG'), ('Python', 'NNP'), ('book', 'NN'), ('written', 'VBN'), ('Akshay', 'NNP')]


[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='5.'></a>5. Cümledeki Sözcüklerden Varlıkları Çıkarma (Named Entity Recognition)

NER metin içerisinde yer alan Adres, Kişi, Organizasyon gibi varlıkları çıkarmaya yarayan doğal dil işleme görevidir.Bunun için nltk kütüphanesinde yer alan ne_chunk fonksiyonu kullanılacaktır.

In [8]:
text = 'John is studying at Stanford University in California'
word_token = word_tokenize(text)
pos_tag = nltk.pos_tag(word_token)
ner  = nltk.ne_chunk(pos_tag, binary = False)
print(ner)

(S
  (PERSON John/NNP)
  is/VBZ
  studying/VBG
  at/IN
  (ORGANIZATION Stanford/NNP University/NNP)
  in/IN
  (GPE California/NNP))


[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='6.'></a>6. Başlık Belirleme

Başlık belirleme dökümanları konularına göre ayırmak için kullanılan doğal dil işleme görevidir. 

In [9]:
class TextProcessor:
    def __init__(self, processors):
        self.processors = processors
    
    def process(self, text):
        for processor in self.processors:
            text = processor.process(text)
        return text

class Lower:
    def process(self, text):
        return text.lower()
    
class FilterPunctuation:
    def __init__(self):
        self.punctuations = string.punctuation
    def process(self, text):
        return ''.join([c for c in text if c not in self.punctuations])

class FilterStopWords:
    def __init__(self, lang):
        self.stopwords = stopwords.words(lang)
    def process(self, text):
        return ' '.join([word for word in text.split() if not word in self.stopwords])
class Lemmatize:
    def __init__(self):
        self.lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    
    def process(self, text):
        return ' '.join([self.lemmatizer.lemmatize(word) for word in text.split()])
text = 'Ohh! I can not belive that!@ cars# is amazing!'

processors = [FilterPunctuation(), 
              FilterStopWords(lang = 'english'),
             Lemmatize()]
textProcessor = TextProcessor(processors = processors)
textProcessor.process(text)

'Ohh I belive car amazing'

In [10]:
doc1 = "I am learning NLP, it is very interesting and exciting. it includes machine learning staffs and deep learning reciepts"
doc2 = "My father is a data scientist and he is nlp expert"
doc3 = "My sister has goods exposure into android development"
docs = [doc1, doc2, doc3]
docs_clean = [textProcessor.process(text).split() for text in docs]
print(docs_clean)

[['I', 'learning', 'NLP', 'interesting', 'exciting', 'includes', 'machine', 'learning', 'staff', 'deep', 'learning', 'reciepts'], ['My', 'father', 'data', 'scientist', 'nlp', 'expert'], ['My', 'sister', 'good', 'exposure', 'android', 'development']]


In [11]:
dictionary = corpora.Dictionary(docs_clean)
doc_term_matrix  = [dictionary.doc2bow(doc) for doc in docs_clean]
print(doc_term_matrix)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1)], [(10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(10, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]


In [12]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics = 3, id2word = dictionary, passes = 50)
print(ldamodel.print_topics())

[(0, '0.048*"good" + 0.048*"exposure" + 0.048*"development" + 0.048*"android" + 0.048*"scientist" + 0.048*"nlp" + 0.048*"data" + 0.048*"sister" + 0.048*"father" + 0.048*"expert"'), (1, '0.123*"My" + 0.070*"data" + 0.070*"nlp" + 0.070*"father" + 0.070*"expert" + 0.070*"sister" + 0.070*"development" + 0.070*"exposure" + 0.070*"android" + 0.070*"scientist"'), (2, '0.175*"learning" + 0.070*"NLP" + 0.070*"deep" + 0.070*"exciting" + 0.070*"includes" + 0.070*"interesting" + 0.070*"I" + 0.070*"machine" + 0.070*"reciepts" + 0.070*"staff"')]


[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='7.'></a> 7. Metin Sınıflandırma

In [15]:
import numpy as np
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
from nltk.stem import PorterStemmer
porterStemmer = PorterStemmer()
class WordStemmer:
    def ___init__(self):
        self.stemmer = PorterStemmer()
    
    def process(self, text):
        stemmer = PorterStemmer()
        return ' '.join([stemmer.stem(word) for word in text.split()])

dataset = pd.read_csv('datasets/spam.csv', encoding = 'latin1')
dataset = dataset[['v2', 'v1']]
dataset.columns = ['email', 'target']

X = dataset['email'].values
y = dataset['target'].values

processors = [Lower(),
              FilterPunctuation(), 
              FilterStopWords(lang = 'english'),
             Lemmatize()]

textProcessor = TextProcessor(processors)
X = [textProcessor.process(x) for x in X]
le = LabelEncoder()
le.fit(y)
y = le.transform(y)
classes = le.inverse_transform(np.arange(len(np.unique(y))))
print('classes:', classes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = MultinomialNB(alpha = 0.2)
clf.fit(X_train_tfidf, y_train)
print('Eğitim başarı oranı:{:.2f}'.format(100*clf.score(X_train_tfidf, y_train)))
print('Test başarı oranı:{:.2f}'.format(100*clf.score(X_test_tfidf, y_test)))

classes: ['ham' 'spam']
Eğitim başarı oranı:99.56
Test başarı oranı:97.91


[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='8.'></a> 8. Duygu Analizi (TextBlob Kütüphanesiyle)

In [22]:
review1 = 'This computer is really disappointment for me. lack of art'
review2 = 'None can find better than this tv game'

for review in [review1, review2]:
    textBlob = TextBlob(review)
    print(textBlob.sentiment)

Sentiment(polarity=-0.6, subjectivity=0.4)
Sentiment(polarity=0.04999999999999999, subjectivity=0.45)


[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='9.'></a>9. Belirsizlik Giderme  

[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='10.'></a>10. Konuşmayı Metne Dönüştürme

In [17]:
import speech_recognition as sr

In [18]:
r=sr.Recognizer()
with sr.Microphone() as source:
    print("Please say something")
    audio = r.listen(source)
    print("Time over, thanks")
try:
    print("I think you said: "+r.recognize_google(audio, language='tr'));
except:
    print('Malesef bir hata oluştu...')

Please say something
Time over, thanks
I think you said: Merhabalar beni dinliyor musun Google bilmiyorsan söylediklerimi yaz


[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='11.'></a>11. Metni Konuşmaya Dönüştürme

In [24]:
#!pip install gTTS

In [27]:
from gtts import gTTS
convert = gTTS(text = 'Bugün yapacağımız işlemleri anlatıyorum: metin sınıflandırma ve duygu analizi',
              lang = 'tr', slow = False)
convert.save('outputs/yazidan_konusmaya.mp3')

[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id ='12.'></a>12. Tercüme Yapma

In [29]:
#!pip install goslate

In [32]:
import goslate
gs = goslate.Goslate()

text = 'Bugün derste işleyeceğimiz konular şunlardır: metin sınıflandırma ve duygu analizi'
translated_text = gs.translate(text, 'en')
print(translated_text)

Today we'll cover topics in the course include: text classification and sentiment analysis
